In [15]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')

In [16]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))  

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]    
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))    

In [18]:
# parameters
customer_id = '137'
formatted_attribute = 'Material'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Window Blinds"""
buckets = """Package Quantity"""

regex_pattern=r'(?i)(x\s?\d+.?\d*\".?D)|(.?.?.?.?.?Depth.?.?.?.?)()'

value='%n/a%'
customer_name='%kimballmidwest%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
# curated_by=762

In [19]:
# df = query_from_file(file_name='Curate_wipe/BusinessIntelligence/Platform Queries/Bucket Attribute Values for All active Strategies.sql', params=params)
df = query_from_file(file_name='query/Delete_Motion_Max_Min_temp_date.sql', params=params)
# df=df[df['value']=='n/a']
print(len(df))
df.head(1)

0


Empty DataFrame
Columns: [attributes, snake_case_name, external_id, product_id, long_desc, curated_date, customers_id, buckets, value, resolution]
Index: []

In [7]:
kim=pd.read_csv('Kimball.csv')
kim=kim[['external_id','buckets','active','name','description']]
len(kim)

47920

In [8]:
regex_pattern_imp = r"(?i)(\d+\-?\d*\/?\d*\-?\")|(\d+\-?\d*\/\d*\-?\"?)|()" 
# kim['matches'] =kim['description'].apply(lambda x: re_extract(regex_pattern_wall, x))
kim['matches'] =kim['name'].apply(lambda x: re_extract(regex_pattern_imp, x))
kim[kim['matches'].astype(str)!='[]']

external_id                                                   buckets  \
0          871349  Power Sanders & Grinders Accessories & Replacement Parts   
3           87742  Power Sanders & Grinders Accessories & Replacement Parts   
4           87743  Power Sanders & Grinders Accessories & Replacement Parts   
5           87744  Power Sanders & Grinders Accessories & Replacement Parts   
6           87745  Power Sanders & Grinders Accessories & Replacement Parts   
...           ...                                                       ...   
47874      824941              Rotary Tools Accessories & Replacement Parts   
47875      824942              Rotary Tools Accessories & Replacement Parts   
47876      824943              Rotary Tools Accessories & Replacement Parts   
47918      829898                                                       NaN   
47919      829899                                                       NaN   

       active                                       name  \
0        True       #8-32 x 6" Threaded Mounting Mandrel   
3        True              1" Cartridge Mounting Mandrel   
4        True          1-1/2" Cartridge Mounting Mandrel   
5        True              1" Cartridge Mounting Mandrel   
6        True          1-1/2" Cartridge Mounting Mandrel   
...       ...                                        ...   
47874    True           1/2 SP Maintenance Length Reamer   
47875    True              5/8 Maintenance Length Reamer   
47876    True           3/4 SP Maintenance Length Reamer   
47918    True  1/8" Pneumatic Pencil Grinder and Bur Set   
47919    True              1/8" Pneumatic Pencil Grinder   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   description  \
0      CROSS BUFF DESIGN - Multi-ply 3 dimensional construction is ideal for conforming to interior diameters. REACHES HARD-TO-GET-AT AREAS - Polishes and cleans inside diameters of tubing, dies, molds, lifter bores, valve bodies, brake cylinders, and other parts. CLEANS - Removes surface oxidation, rust, corrosion, and contaminants. POLISHES - Provides a smooth and consistent finish. TOUGH NON-WOVEN CONSTRUCTION - Resists loading on softer materials, and runs cool and eliminates edge fraying and grain shedding. SURFACE CONDITIONING - Removes scratches, flashing, rust and oxidation from metal surfaces. Provides light deburring, and blending of ground out welds. NON-GOUGING - Non-woven design eliminates gouging or changing critical tolerances. VERSATILE - Excellent on virtually all metals including steel, aluminum, stainless steel, tool steel, formed surfaces, castings and forgings. QUICK CHANGE SCREW-ON ARBOR - Buffs have an internal female screw thread for fast attachment to the mandrel. MADE IN THE U.S.A.   
3                                                                                                                                                                                                                                                                                           

In [9]:
kim[kim['matches'].astype(str)=='[]']

external_id                                                   buckets  \
1          871461  Power Sanders & Grinders Accessories & Replacement Parts   
2          872255  Power Sanders & Grinders Accessories & Replacement Parts   
8           87955  Power Sanders & Grinders Accessories & Replacement Parts   
9           87830  Power Sanders & Grinders Accessories & Replacement Parts   
10          87831  Power Sanders & Grinders Accessories & Replacement Parts   
...           ...                                                       ...   
47913      824944                                                       NaN   
47914      281832                                                       NaN   
47915     2818321                                                       NaN   
47916     2818322                                                       NaN   
47917     2818323                                                       NaN   

       active                                         name  \
1        True                 4-Flap Bore Polisher Mandrel   
2        True                   Flush Type Mandrel Adapter   
8        True                1â€ Type â€œRâ€ Disc Holder   
9        True              Unitized Wheel Mounting Mandrel   
10       True              Unitized Wheel Mounting Mandrel   
...       ...                                          ...   
47913    True             SP Maintenance Length Reamer Set   
47914    True                          Quantumâ„¢ B.A.R. 2   
47915    True    Replacement Hinge for Quantumâ„¢ B.A.R. 2   
47916    True    Replacement Mount for Quantumâ„¢ B.A.R. 2   
47917    True  Replacement Charger for Quantumâ„¢ B.A.R. 2   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              description  \
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [10]:
regex_pattern_mm = r"(?i)(\d+\.\d*.?mm)|(\d+.?mm)|()" 
kim_mm=kim
kim_mm['new_name_matches'] =kim_mm['name'].apply(lambda x: re_extract(regex_pattern_mm, x))
kim_mm[kim_mm['new_name_matches'].astype(str)!='[]']

external_id                                                   buckets  \
1181        87809  Power Sanders & Grinders Accessories & Replacement Parts   
1193        87812  Power Sanders & Grinders Accessories & Replacement Parts   
1239        87808  Power Sanders & Grinders Accessories & Replacement Parts   
1243        87810  Power Sanders & Grinders Accessories & Replacement Parts   
1244        87816  Power Sanders & Grinders Accessories & Replacement Parts   
...           ...                                                       ...   
47204      844188                                                       NaN   
47205      844189                                                       NaN   
47206      844190                                                       NaN   
47208      844191                                                       NaN   
47209      844192                                                       NaN   

       active                                                 name  \
1181     True                       14" x 1/8 x 20mm Cut-Off Wheel   
1193     True  14" x 1/8 x 20mm Cut-Off Wheel For Gas Powered Saws   
1239     True                       12" x 1/8 x 20mm Cut-Off Wheel   
1243     True  12" x 1/8 x 20mm Cut-Off Wheel For Gas Powered Saws   
1244     True               14" x 1/8 x 20mm Masonry Cut-Off Wheel   
...       ...                                                  ...   
47204    True                        5mm Ball End Power Insert Bit   
47205    True                        6mm Ball End Power Insert Bit   
47206    True                        8mm Ball End Power Insert Bit   
47208    True                       10mm Ball End Power Insert Bit   
47209    True                       12mm Ball End Power Insert Bit   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             description  \
1181                                                                                                                                                                                                                                                                                                                                        Premium reinforced wheels. Use on portable electric saws and chop-type cut-off machines. Use on steels, alloy steels, angle iron, pipe, tubing, and conduit.   
1193                                                                                                                                                                                                                                                                                                          SRT (Synthetic Resinoid Technology) outperforms conventional wheels up to three to one. Maximum life, aggressive cutting. Use on ferrous metals, stainless steel, and high tensile alloys.   
1239                                                                                                                                                                                                                                                                                                                                        Premium reinforced wheels. Use on portable electric saws and chop-type cut-off machines. Use on steels, alloy steels, angle iron, pipe, tubing, and conduit.   
1243                                                                                                                                                                                                                                                    

In [11]:
regex_pattern_mm = r"(?i)(\d+\.\d*.?mm)|()" 
kim_mm=kim
kim_mm['name_matches'] =kim_mm['name'].apply(lambda x: re_extract(regex_pattern_mm, x))
# kim['matches'] =kim['description'].apply(lambda x: re_extract(regex_pattern_wall, x))
kim_mm=kim_mm[kim_mm['name_matches'].astype(str)!='[]']
len(kim_mm)

208

In [12]:
regex_pattern_mm_in = r'(?i)(\d*\.\d*.?mm)|(\d*.?mm)|(\d+\-?\d*\/\d*\-?\"?\s?x\s?\d*\")|(\d*\"\s?x\s?\d+\-?\d*\/\d*\-?\"?)|(\d+\-?\d*\/\d*\-?\"?\s?x\s?\d+\-?\d*\/\d*\-?\"?)|(\d+\-?\d*\/\d*\-?\"?)|([^\/][^\)\.]\d*\")|()'
kim['non_matches'] =kim['name'].apply(lambda x: re_extract(regex_pattern_mm, x))
kim[kim['non_matches'].astype(str)=='[]']

external_id                                                   buckets  \
0          871349  Power Sanders & Grinders Accessories & Replacement Parts   
1          871461  Power Sanders & Grinders Accessories & Replacement Parts   
2          872255  Power Sanders & Grinders Accessories & Replacement Parts   
3           87742  Power Sanders & Grinders Accessories & Replacement Parts   
4           87743  Power Sanders & Grinders Accessories & Replacement Parts   
...           ...                                                       ...   
47915     2818321                                                       NaN   
47916     2818322                                                       NaN   
47917     2818323                                                       NaN   
47918      829898                                                       NaN   
47919      829899                                                       NaN   

       active                                         name  \
0        True         #8-32 x 6" Threaded Mounting Mandrel   
1        True                 4-Flap Bore Polisher Mandrel   
2        True                   Flush Type Mandrel Adapter   
3        True                1" Cartridge Mounting Mandrel   
4        True            1-1/2" Cartridge Mounting Mandrel   
...       ...                                          ...   
47915    True    Replacement Hinge for Quantumâ„¢ B.A.R. 2   
47916    True    Replacement Mount for Quantumâ„¢ B.A.R. 2   
47917    True  Replacement Charger for Quantumâ„¢ B.A.R. 2   
47918    True    1/8" Pneumatic Pencil Grinder and Bur Set   
47919    True                1/8" Pneumatic Pencil Grinder   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              description  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                 CROSS BUFF DESIGN - Multi-ply 3 dimensional construction is ideal for conforming to interior diameters. REACHES HARD-TO-GET-AT AREAS - Polishes and cleans inside diameters of tubing, dies, molds, lifter bores, valve bodies, brake cylinders, and other parts. CLEANS - Removes surface oxidation, rust, corrosion, and contaminants. POLISHES - Provides a smooth and consistent finish. TOUGH NON-WOVEN CONSTRUCTION - Resists loadi

In [13]:
kim['buckets'].explode().value_counts()

Fixings & Fasteners                                                      14789
Power & Impact Drills/Drivers/Presses Accessories & Replacement Parts     1553
Power Sanders & Grinders Accessories & Replacement Parts                  1504
Cables, Cords & Cord Management Replacement Parts & Accessories           1075
Taps & Dies                                                                870
Fuses & Circuit Breakers                                                   848
Rotary Tools Accessories & Replacement Parts                               665
Cables, Cords, & Adapters                                                  426
Vehicle Lighting & Light Bulbs                                             391
Paint, Stains, & Varnishes                                                 326
Conduit Fittings, Replacement Parts, & Accessories                         269
Sealants                                                                   262
Sockets & Ratchets                                  

# Seperate Bukets and start curating

# Wall Sculptures

In [7]:
wall=df[df['buckets'].astype(str)=='Wall Sculptures & Dimensional Art']
regex_pattern_wall = r"(?i)(\d*\.?\d*.?Inch\s?x\s?\d*\.?\d*.?Inch)|(\d*.?Inch)|(\d*\.\d*.?Inch)|()" 
wall['matches'] =wall['long_desc'].apply(lambda x: re_extract(regex_pattern_wall, x))
wall['name_matches'] =wall['product_name'].apply(lambda x: re_extract(regex_pattern_wall, x))
wall_df=wall[wall['name_matches'].astype(str)!='[]']

# Square

In [8]:
wall_df['name_matches']=wall_df['name_matches'].str.get(0)
wall_df[['length', 'width']]=wall_df['name_matches'].str.split("x",expand=True,)
wall_df.head(1)

product_id external_id  \
145  1230264608    69724775   

                                                                       product_name  \
145  Glitzhome® 24.25-Inch x 11.25-Inch Whimsical Metal Watermelon Gecko Wall Decor   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                             long_desc  \
145  Add a fun look to your wall with this Whimsical Metal Watermelon Gecko Wall Decor from Glitzhome. Crafted of sturdy iron to last, this colorful wall decor is ready to hang and displays a cute gecko with watermelon coloring and big pink eyes. Metal gecko shape Spring neck Displays watermelon design elements for a cute look Hand painted glossy finish Appropriate for indoor/outdoor use Iron Ready to hang Weighs 1 lb. 1-year limited manufacturer's warranty Imported   

     customer_id attributes                            buckets  bucket_id  \
145            5       None  Wall Sculptures & Dimensional Art       8122   

    values matches             name_matches       length        width  
145   None      []  24.25-Inch x 11.25-Inch  24.25-Inch    11.25-Inch

In [9]:
wall_df['length']=wall_df['length'].apply(lambda measure: re.sub(r"\-Inch",'',str(measure)))
wall_df['width']=wall_df['width'].apply(lambda measure: re.sub(r"\-Inch",'',str(measure)))  
wall_df['length']=wall_df['length'].apply(lambda measure: re.sub(r" ",'',str(measure)))
wall_df['width']=wall_df['width'].apply(lambda measure: re.sub(r" ",'',str(measure))) 
wall_df['length']=wall_df['length'].astype(str)
wall_df['width']=wall_df['width'].astype(str)

In [10]:
wall_df['compared']=wall_df['length'].equals(wall_df['width'])
wall_df.loc[(wall_df['length']==wall_df['width']), 'compared']=wall_df['length']
wall_new=wall_df[wall_df['compared'].astype(str)!='False']
wall_new[curation_col]=wall_new['compared']   
wall_new[curation_col]=wall_new[curation_col].apply(lambda x: x+' in')
wall_new.head(1)

In [69]:
len(wall_new)

67

In [12]:
square=wall_df[wall_df['width'].astype(str)=='None']
square[curation_col]=square['length']
square[curation_col]=square[curation_col].apply(lambda x: x+' in')
square.head(1)

product_id external_id  \
240  1016177218    69674934   

                                                 product_name  \
240  28.5-Inch Metal Wind Spinner Wall Decor in Vintage White   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               long_desc  \
240  Crafted of galvanized iron, the Wind Spinner Wall Décor adds a farmhouse touch to any wall. With quiet vintage appeal, this 3D fan shaped design has rivets and flanging on both sides for added depth, and a rustic look. Adds farmhouse charm to any home décor 3D fan-shaped design with rivets added on Flanging on both sides give the decor depth Vintage red, vintage white and grey painted finishes 100% iron Ready to hang with gourd hole on the back Measures 22\" diameter or 28.5\" diameter Imported   

     customer_id attributes                            buckets  bucket_id  \
240            5       None  Wall Sculptures & Dimensional Art       8122   

    values matches name_matches length width compared Q:height  
240   None      []    28.5-Inch   28.5  None    False  28.5 in

In [68]:
len(square)

244

# Trees

In [43]:
trees=df[df['buckets'].astype(str)=='Trees, Shrubs, Topiaries, & Other Plants']
# \d*.?Foot
regex_pattern_tree = r"(?i)(\b\d*.?Foot\s?\d+.?Inch)|(\b\d*.?\d*\/\d*.?Foot)|(\b\d*\.?\d*.?Foot)|(\b\d*.?Foot)|()" 
trees['matches'] =trees['long_desc'].apply(lambda x: re_extract(regex_pattern_tree, x))
trees['name_matches'] =trees['product_name'].apply(lambda x: re_extract(regex_pattern_tree, x))
trees_df=trees[trees['name_matches'].astype(str)!='[]']

In [46]:
#import regex as re
matches_tree=trees_df
matches_tree[curation_col]=matches_tree['name_matches'].astype(str)
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda x: f'"{x}"')
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"\-Foot",' ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r'\-foot',' ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"\-1\/2",'.5',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r' 1\/2','.5',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r" Foot",' ft',str(measure)))

matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r'7 ft 6\-Inch','7.5 ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"6 ft 5\-Inch",'6.4 ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r'4 ft 6\-Inch','4.6 ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"7 ft 5\-Inch",'7.4 ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"2 ft 5\-Inch",'2.4 ft',str(measure)))

matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
matches_tree.head(10)

product_id external_id  \
60     817488197    47362154   
77    1199645354    45485725   
78     817169202    45485633   
1606  1174458301    66735656   
2413   817266290    62508643   
2414   817266252    62510691   
2415   817266253    62505772   
2416   817530699    62507837   
2418   817530639    62505987   
2419   817530590    62486071   

                                                                                        product_name  \
60                            National Tree Company 3-Foot Pine Cone Tree with Warm White LED Lights   
77                           Fraser Hill Farm 7.5-Foot Southern Peace Pine Artificial Christmas Tree   
78                    Fraser Hill Farm 7.5-Foot Hillside Slim Flocked Pine Artificial Christmas Tree   
1606                National Tree Company Dual Color® 4.5-Foot Pre-Lit Newbury Spruce Christmas Tree   
2413                          National Tree Company 7-Foot Fraser Slim Fir Artificial Christmas Tree   
2414    National Tree Company 7-1/2-Foot Pre-Lit North Valley White Spruce Artificial Christmas Tree   
2415    National Tree Company 4-1/2-Foot Pre-Lit North Valley White Spruce Artificial Christmas Tree   
2416                           National Tree Company 3-Foot Kincaid Spruce Artificial Christmas Tree   
2418  National Tree Company® 6.5-Foot Downswept Douglas Pencil Slim Christmas Tree with Clear Lights   
2419                National Tree Company 6-1/2-Foot Pre-Lit Carolina Pine Artificial Christmas Tree   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         long_desc  \
60                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                The festive Pine Cone Holiday Collection from National Tree Company brings the beauty of the holiday season into your home. These gorgeous holiday pieces bedecked with decorative pinecones and can be displayed indoors or covered outdoor areas. National Tree Company Pine Cone Holiday Collection offers attractive festive pieces to decorate your home indoors or outdoors PVC construction trimmed with decorative pine cones For indoor or covered outdoor use Imported   
77                                                                                                                              

In [47]:
matches_tree[curation_col].explode().value_counts()

["7.5 ft"]    44
["9 ft"]      34
["6.5 ft"]    26
["4.5 ft"]    15
["12 ft"]     13
["3 ft"]      13
["6 ft"]      12
["4 ft"]      11
["5 ft"]      10
["7 ft"]       8
["10 ft"]      5
["2 ft"]       5
["16 ft"]      3
["5.5 ft"]     3
["14 ft"]      2
["15 ft"]      1
["6.4 ft"]     1
["3.5 ft"]     1
["8 ft"]       1
["2.4 ft"]     1
["4.6 ft"]     1
["7.4 ft"]     1
Name: Q:height, dtype: int64

In [67]:
len(matches_tree)

211

# Stools

In [50]:
stools=df[df['buckets'].astype(str)=='Stools']
regex_pattern_stool = r"(?i)(\d*\.\d*.?Inch)|(\d*.?Inch)|(\d.?.?H)|()" 
stools['matches'] =stools['long_desc'].apply(lambda x: re_extract(regex_pattern_stool, x))
stools['name_matches'] =stools['product_name'].apply(lambda x: re_extract(regex_pattern_stool, x))
stools_df=stools[stools['name_matches'].astype(str)!='[]']

In [51]:
#import regex as re
matches_stool=stools_df
matches_stool[curation_col]=matches_stool['name_matches'].astype(str)
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda x: f'"{x}"')
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda measure: re.sub(r"\-Inch",' in',str(measure)))
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda measure: re.sub(r" Inch",' in',str(measure)))
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
matches_stool.head(10)

product_id external_id  \
1653   817330983    66220954   
1755   988656011    66027362   
2579   817259945    61757912   
2580   817259946    61757936   
2874   817229475    45421525   
2875   817229476    45421549   
2926   988579392    18776588   
2944   988574450    40527833   
4161  1244996285    66220183   
6394   817216802    18078465   

                                                                      product_name  \
1653                                        Linon Home Mossy Oak 45-Inch Bar Stool   
1755              American Woodcrafters Taranto 34-Inch Leather Bar Stool in Brown   
2579    Carolina Cottage Remick 24-Inch Barley Twist Leg Counter Stool in Espresso   
2580  Carolina Cottage Remick 30-Inch Barley Twist Leg Bar Stool in Espresso/Cream   
2874                               Westwood Patches 24-Inch Counter Stool in White   
2875                                             Westwood 24-Inch Bar Stool in Red   
2926                      Crosley X-Back 24-Inch Counter Stool in Black (Set of 2)   
2944                                           Lakewood 30-Inch Bar Stool in Brown   
4161                             Linon Home Westwood 26-Inch Counter Stool in Grey   
6394                                               Hillsdale Jenkins 27-Inch Stool   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
1653                                                                                                                                                                                                                                                                              Add a rustic look to your home with the Linon Mossy Oak Bar and Counter Stools. Featuring a dark walnut finish and a unique nature inspired print upholstery, these classic stools have four foot rails for added comfort and stability. Adds rustic style to your home counter or bar Dark walnut finish is accented with hand brushed black strokes for depth Plush padded upholstery Unique Mossy Oak Nativ Living print Four footrails add stability and durability to the stool Wood/fabric/foam Spot clean 275 lb. weight capacity 6-year limited manufacturer's warranty Imported   
1755                                                                                                                                                                                                                                                                                                                      The Taranto Stool features industrial flare with its charcoal finished steel back and solid wood cap accent. The washed brown finish and flared legs add style and are accented by the black bonded leather seat and charcoal finished nailhead trim. A 360 degree swivel keeps you in the conversation while the circular footrest ensures your comfort Stylish charcoal finished nail head accents Assembly required Leather/foam seat Wood frame construction 250 lb. maximum weight capacity 1-year limited manufacturer's warranty Imported   
2579                                                           Featuring spiral barley-twist legs, soft cotton-linen upholstery and brass nailhead trim, the striking Re

In [52]:
matches_stool[curation_col].explode().value_counts()

["30 in"]       25
["24 in"]       19
["26 in"]        5
["29 in"]        5
["25 in"]        3
["31 in"]        3
["32 in"]        2
["34 in"]        2
["44.5 in"]      2
["26.4 in"]      2
["26.5 in"]      1
["27 in"]        1
["38.75 in"]     1
["41 in"]        1
["39 in"]        1
["18 in"]        1
["20 in"]        1
["30.5 in"]      1
["45 in"]        1
["24.5 in"]      1
Name: Q:height, dtype: int64

In [66]:
len(matches_stool)

78

# Side Tables

In [61]:
side_tables=df[df['buckets'].astype(str)=='Side/End Tables']
regex_pattern_table = r"(?i)(\d*\.\d*”.?H)|(\d*\.\d*.?Inch)|(\d*.?Inch)|(\d*”.?H)|()" 
side_tables['matches'] =side_tables['long_desc'].apply(lambda x: re_extract(regex_pattern_table, x))
side_tables['name_matches'] =side_tables['product_name'].apply(lambda x: re_extract(regex_pattern_table, x))
side_tables_df=side_tables[side_tables['matches'].astype(str)!='[]']
side_tables_df['matches']= side_tables_df['matches'].apply(lambda x: sorted(x, key=natural_key))

In [62]:
#import regex as re
matches_side_table=side_tables_df
matches_side_table[curation_col]=side_tables_df['matches'].astype(str)
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda x: f'"{x}"')
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r'\” H',' in',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r'\”H',' in',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r" ",'',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r"in",' in',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))

In [65]:
len(matches_side_table)

90

# Sculptures and figurines

In [126]:
sculptures=df[df['buckets'].astype(str)=='Sculptures, Figurines, & Decorative Objects']
# \d*.?Foot
regex_pattern_sc = r"(?i)(\b\d*.?Foot\s?\d+.?Inch)|(\b\d*.?\d*\/\d*.?Foot)|(\b\d*\.?\d*.?Foot)|(\b\d*.?Foot)|(\d*\.\d*”.?H)|(\d*\.\d*.?Inch.?H)|(\d*.?Inch.?H)|(\d*”.?H)|()" 
sculptures['matches'] =sculptures['long_desc'].apply(lambda x: re_extract(regex_pattern_sc, x))
sculptures['name_matches'] =sculptures['product_name'].apply(lambda x: re_extract(regex_pattern_sc, x))
sculptures_df=sculptures[(sculptures['matches'].astype(str)!='[]')]
sculptures_df=sculptures_df[sculptures_df['external_id'].astype(str)!='69241383']
sculptures_df=sculptures_df[sculptures_df['external_id'].astype(str)!='16623601']
sculptures_df['matches']= sculptures_df['matches'].apply(lambda x: sorted(x, key=natural_key))
# sculptures_df

In [128]:
regex_pattern_sc_mis = r"(?i)(\b\d*.?Foot\s?\d+.?Inch)|(\b\d*.?\d*\/\d*.?Foot)|(\b\d*\.?\d*.?Foot)|(\b\d*.?Foot)|(\d*\.\d*''.?H)|(\d*\.\d*.?Inch.?H)|(\d*.?Inch.?H)|(\d*''.?H)|()" 
sculptures['mis_matches'] =sculptures['long_desc'].apply(lambda x: re_extract(regex_pattern_sc_mis, x))
sculptures['misc_name_matches'] =sculptures['product_name'].apply(lambda x: re_extract(regex_pattern_sc_mis, x))
sculptures_df=sculptures[(sculptures['misc_name_matches'].astype(str)!='[]')]
sculptures_df=sculptures_df[sculptures_df['external_id'].astype(str)!='69241383']
sculptures_df=sculptures_df[sculptures_df['external_id'].astype(str)!='16623601']
sculptures_df['matches']= sculptures_df['matches'].apply(lambda x: sorted(x, key=natural_key))
sculptures_df

product_id external_id  \
6404   1027974529    65988633   
23624  1196481650    65994351   
25008  1011989107    69406447   
38221  1174818157    69606258   
61918  1071790838    69406430   
85552   988764235    69623246   

                                                                              product_name  \
6404      5-Foot Standing Santa Claus Christmas Figure with Flocked Alpine Tree and Wreath   
23624                                Vickerman 6-Foot Plush Scarecrow Decoration in Purple   
25008  Northlight 5-Foot Snowflake Silhouette LED Rope Light Christmas Decoration in White   
38221                           5-Foot Skeleton Unicorn Inflatable LED Lit Lawn Decoration   
61918  Northlight 4-Foot Snowflake Silhouette LED Rope Light Christmas Decoration in White   
85552                                                Mr. Pumpkin 3-Foot Metal Candy Holder   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            long_desc  \
6404                                                                                                                                                                                                                                                                                            Santa Claus is coming to town. Bring a rustic feel to your home this Christmas season with this standing Santa Claus figure. Wearing a plaid fur-trimmed coat, hat and matching boots, he is a unique but festive touch to your holiday decor. Smiling Santa wearing glasses Santa is wearing a red and black coat with brown faux fur trim He is carrying a wreath in one hand and a flocked tree in the other Fabric/plastic/faux fur/wood Measures 5' H x 22\" W x 24\" D Imported   
23624                                                                                                                                                                                                                                                          Add a classic, festive accent to your home with Vickerman's 6' Plush Autumn Scarecrow. Perfect to place in your home for the Fall season, this scarecrow can easily stand up with the PVC pole frame or be adjusted into a sitting position. Stands 6 feet tall! Easy assembly required Plush parts connect with PVC poles to make the scarecrow stand up or sit down Scarecrow's feet are connected to a 16\" x 16\" display platform This is a display figure it is not intended to be used as a toy Indoor use only   
25008             This Northlight Snowflake Silhouette LED Rope Light Christmas Decoration is perfect for adding seasonal charm to your space for the holidays. Ready-to- hang snowflake has a dual sided design with clear LED lights that cast a cozy, festive glow. Illuminated dual-sided snowflake silhouette 1.5\" bulb spacing Suitable for indoor/outdoor use 2' snowflake features: 120 clear LED bulbs Measures 24' L x 24\" W Weighs 2 lb. 4' snowflake features: 216 clear LED bulbs Measures 48\" L x 48\" W Weighs 2.8 lb. 5' snowflake features: 264 clear LED bulbs Measures 60\" L x 60\" W Weighs 2.5 lb Plastic/metal/ wire Plugs into standard outlets Hangable design No assembly required White lead cord measures 6' L UL listed Imported Each sold separately   
38221  Add spooky Halloween fun and whimsy to your yard with this Skelet

In [121]:
#import regex as re
matches_sc=sculptures_df
matches_sc[curation_col]=sculptures_df['matches'].astype(str)

matches_sc[curation_col]=matches_sc[curation_col].apply(lambda x: f'"{x}"')
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'\”','',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'H',' in',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'  ',' ',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'Inch H',' in',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'Inch','',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"\-",'',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r' ','',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"in",' in',str(measure)))

matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'1 in","16 in','16 in',str(measure)))
# matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"6 ft 5\-Inch",'6.4 ft',str(measure)))
# matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'4 ft 6\-Inch','4.6 ft',str(measure)))
# matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"7 ft 5\-Inch",'7.4 ft',str(measure)))
# matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"2 ft 5\-Inch",'2.4 ft',str(measure)))

matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches_sc.head(10)

In [122]:
matches_sc[curation_col].explode().value_counts()

["9 in"]                      4
["14 in"]                     3
["8 in"]                      3
["11 in"]                     3
["12 in"]                     2
["23 in"]                     2
["16 in"]                     2
["5 in"]                      2
["19.5 in"]                   1
["14.5 in"]                   1
["26 in"]                     1
["9 in","10 in"]              1
["19 in"]                     1
["18.5 in"]                   1
["3 in","4 in","5 in"]        1
["20 in"]                     1
["17 in"]                     1
["5.5 in","7 in","8.5 in"]    1
["9.5 in","10 in"]            1
["12.75 in","13 in"]          1
["9.5 in"]                    1
["6.25 in","6 in"]            1
["32 in"]                     1
["13 in"]                     1
["7 in"]                      1
["16 in","17 in"]             1
["12 in","14 in"]             1
["21 in"]                     1
["28 in"]                     1
["15 in"]                     1
["16.5 in"]                   1
["10.75 

In [104]:
matches_sc[matches_sc['matches'].astype(str)==' foot']

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, bucket_id, values, matches, name_matches, Q:height]
Index: []

In [95]:
len(matches_sc)

52

# foot

In [ ]:
# regex_pattern = r"(?i)(\b\d+\.\d*.?\"\s?H\s)|(\b\d+.?\"\s?H)|(\b\d+\.\d*.?\"\s?H)|(\d*\s?\-\d*\/\d*\\\"\s?H)|(\d*\s\-?\d*\/\d*\\\"\s?H)|()" 
# regex_pattern_diam = r'(?i)(\d+\.\d*\s?\\\"\s?i?n?\s?diam)|(\d*\s?\\\"\s?i?n?\s?diam)|()'
# regex_pattern_foot = r"(?i)(\b\d+\.\d*\'\s?H\s)|(\b\d+.?\'\s?H)|(\b\d+\.\d*\'\s?H)|(\d*\s?\-\d*\/\d*\\?\'\s?H)|(\d*\s\-?\d*\/\d*\\\'\s?H)|()" 

In [ ]:
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# df['diam_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diam, x))
# df['foot_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_foot, x))
# len(df)

In [ ]:
# foot=df[df['foot_matches'].astype(str)!='[]']
# foot=foot[foot['buckets'].astype(str)=='Trees, Shrubs, Topiaries, & Other Plants']
# len(foot)

In [ ]:
# # x=df[df['matches'].astype(str)!='[]']
# x=df
# len(x)
# # len(x)

In [ ]:
# x=x[x['matches'].astype(str)!='[]']

In [ ]:
# replacement_foot = {
#     "\' H": " ft",
#     "' H": "ft",
#     '  ft': " ft",
#     "\'H": " ft",
#     "' h": " ft",
#     "3\' H": "3 ft"
# }

In [ ]:
# # Replacement
# foot['foot_matches']= foot['foot_matches'].apply(
#     lambda x: [replace_all(string, replacement_foot) for string in x] if type(x)==list else x)

In [ ]:
# # Replacement
# foot['foot_matches']= foot['foot_matches'].apply(
#     lambda x: [replace_all(string, replacement_foot) for string in x] if type(x)==list else x)
# # foot.head(3)

In [ ]:
# foot['foot_matches']= foot['foot_matches'].apply(
#     lambda x: sorted(x, key=natural_key))
# # foot.head(3)

In [ ]:
# #import regex as re
# matches_foot=foot
# matches_foot[curation_col]=matches_foot['foot_matches'].astype(str)
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda x: f'"{x}"')
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches_foot.head(3)

In [ ]:
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r"\-1\/2",'.5',str(measure)))
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'\"\\xa0H',' ft',str(measure)))

In [ ]:
# matches_foot[curation_col].explode().value_counts()

In [ ]:
replacement = {
    '\" H ': " in",
    "\ in": " in",
    '\" H': " in",
    '\" H': " in",
    '\" H ': " in",
    "\ in": " in",
    '\" H': " in",
    '\" H': " in",
    '\" h': " in",
    '\"H': " in"
}

In [ ]:
x=x[x['matches'].astype(str)!='[]']
x.head(1)

In [ ]:
# Replacement
x['matches']= x['matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

In [ ]:
# Replacement
x['matches']= x['matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)
x.head(1)

In [ ]:
x['matches']= x['matches'].apply(
    lambda x: sorted(x, key=natural_key))
x.head(1)

In [ ]:
# x=x[(x['matches'].astype(str)!='3/4 in')&(x['matches'].astype(str)!='1/4 in')&(x['matches'].astype(str)!='1/5 in')&(x['matches'].astype(str)!=' 3/4 in')]

In [ ]:
# replacement_diam = {
#     'diam': "",
#     " ": "",
#     '\"': "",
#     'in': "",
#     'Diam': ""
# }
# # Replacement
# diam['diam_matches']= diam['diam_matches'].apply(
#     lambda x: [replace_all(string, replacement_diam) for string in x] if type(x)==list else x)

# # Replacement
# diam['diam_matches']= diam['diam_matches'].apply(
#     lambda x: [replace_all(string, replacement_diam) for string in x] if type(x)==list else x)
# diam.head(3)

# diam['diam_matches']= diam['diam_matches'].apply(
#     lambda x: sorted(x, key=natural_key))
# diam.head(3)

# matches

In [ ]:
# matches=x

In [ ]:
# #import regex as re
# matches[curation_col]=matches['matches'].astype(str)
# matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches.head(3)

In [ ]:
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1\/2','.5',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-3\/4','.75',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 3\/4','.75',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1\/2','.5',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1\/4','.25',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1\/4','.25',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-3\/8','.38',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 3\/8','.38',str(measure)))

# matches=matches[matches[curation_col].astype(str)!='["3/4 in"]']
# matches=matches[matches[curation_col].astype(str)!='["1/4 in"]']
# matches=matches[matches[curation_col].astype(str)!='["1/5 in"]']

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-4\/5','.8',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 4\/5','.8',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1/5','.2',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1/5','.2',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-5/8','.62',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 5/8','.62',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-2/3','.66',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 2/3','.66',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-3/10','.3',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 3/10','.3',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1/3','.33',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1/3','.33',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-8/9','.89',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 8/9','.89',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-3/5','.6',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 3/5','.6',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-2/5','.4',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 2/5','.4',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-7/10','.7',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 7/10','.7',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1/10','.1',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1/10','.1',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1/8','.13',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1/8','.13',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-7/8','.88',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 7/8','.88',str(measure)))

In [ ]:
# # matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'1575 in','15.75 in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\\xa0','',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'4563 in','45.63 in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'3625 in','36.25 in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\.625','.63',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\.375','.38',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\.875','.88',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'335 in","635 in','33.5 in","63.5 in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'45 in","395 in','45 in","39.5 in',str(measure)))

In [ ]:
# matches[matches[curation_col].astype(str)=='["0.59 in"]']

In [ ]:
# matches[matches[curation_col].astype(str)=='["45 in","395 in"]']

In [ ]:
len(matches)

In [ ]:
# matches_diam=diam

In [ ]:
# #import regex as re
# matches_diam[curation_col]=matches_diam['diam_matches'].astype(str)
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda x: f'"{x}"')
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches_diam.head(3)

In [ ]:
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\\\\',' in',str(measure)))

In [ ]:
# matches_diam=matches_diam[matches_diam['buckets'].astype(str)!='Trees, Shrubs, Topiaries, & Other Plants']
# matches_diam=matches_diam[matches_diam['buckets'].astype(str)=='Hanging Accents & Accessories']

In [ ]:
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\\',' in',str(measure)))

In [ ]:
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'.875','.88',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\"10 in\"\,\"10 in\"','"10 in"',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'.725','.73',str(measure)))

In [ ]:
# matches_diam[curation_col].explode().value_counts()[0:500]

In [ ]:
# error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [123]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets-height-sculpture - {attribute}-- match upload_Shawn.csv',index=False)

In [124]:
looks_good('Bed Bath & Beyond', attribute, df, matches_sc)

In [ ]:
def looks_good_na(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets-NA - {attribute}-- match upload_Shawn.csv',index=False)

In [ ]:
looks_good_na('Bed Bath & Beyond', attribute, df, na_upload)

In [ ]:
matches_side_table+matches_stool+matches_tree